In [9]:
# Import libraries and dependencies
import os
import numpy as np
import pandas as pd
import alpaca_trade_api as tradeapi
#from MCForecastTools import MCSimulation
import plotly.express as px
import panel as pn
import hvplot.pandas
import pypfopt
from pypfopt import risk_models, expected_returns, plotting
from pypfopt import expected_returns
%matplotlib inline

In [10]:
# Load .env enviroment variables
from dotenv import load_dotenv
load_dotenv()

# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

api = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version = "v2"
)

In [11]:
# Set the ticker "AAPL","MSFT","AMZN","GOOG","FB","TSLA","BRK.A","V","JPM","JNJ". These are the top 10 US stocks on the NYSE by MarketCap found on  stock found http://www.iweblists.com/us/commerce/MarketCapitalization.html
ticker =["AAPL","MSFT","AMZN","GOOG","FB","TSLA","BRK.A","V","JPM","JNJ"]

# Set timeframe to '1D'
timeframe = "1D"

# Set start and end datetimes of 2 years from Today
start_date = pd.Timestamp("2019-04-20", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2021-04-20", tz="America/New_York").isoformat()

# Get 1 year's worth of historical data for "AAPL","MSFT","AMZN","GOOG","FB","TSLA","BRK.A","V","JPM","JNJ".
df_portfolio_year = api.get_barset(
    ticker,
    timeframe,
    start=start_date,
    end=end_date
).df

df_portfolio_year.head()
ticker =["SPY"]

# Set timeframe to '1D'
timeframe = "1D"

#year data of the top 10 highest market cap
df_portfolio_year.head()

AAPL                                      \
                             open    high     low    close     volume   
time                                                                    
2020-10-28 00:00:00-04:00  115.05  115.43  111.10  111.150  123242639   
2020-10-29 00:00:00-04:00  112.37  116.93  112.20  114.520  111552689   
2020-10-30 00:00:00-04:00  111.06  111.99  107.72  108.900  163824978   
2020-11-02 00:00:00-05:00  109.11  110.68  107.32  108.770  106260834   
2020-11-03 00:00:00-05:00  109.66  111.47  108.73  110.375   93660131   

                              AMZN                                        ...  \
                              open      high       low    close   volume  ...   
time                                                                      ...   
2020-10-28 00:00:00-04:00  3249.29  3264.020  3162.470  3162.49  4840893  ...   
2020-10-29 00:00:00-04:00  3201.27  3257.250  3164.000  3212.79  5352688  ...   
2020-10-30 00:00:00-04:00  3157.75  3167.004  3019.000  3036.13  7556383  ...   
2020-11-02 00:00:00-05:00  3061.74  3079.705  2950.395  3004.48  6469197  ...   
2020-11-03 00:00:00-05:00  3018.53  3074.900  2980.980  3046.25  4399819  ...   

                               TSLA                                      \
                               open      high     low   close    volume   
time                                                                      
2020-10-28 00:00:00-04:00  416.4800  418.6000  406.00  406.00  23252693   
2020-10-29 00:00:00-04:00  409.9600  418.0600  406.46  410.64  20826471   
2020-10-30 00:00:00-04:00  406.8954  407.5915  379.11  388.04  40305209   
2020-11-02 00:00:00-05:00  394.0000  406.9799  392.30  400.51  26759798   
2020-11-03 00:00:00-05:00  409.7300  427.7700  406.69  423.74  32428696   

                                V                                    
                             open    high     low   close    volume  
time                                                                 
2020-10-28 00:00:00-04:00  183.98  184.70  179.34  181.35  30162189  
2020-10-29 00:00:00-04:00  183.00  187.00  179.34  184.86  26874224  
2020-10-30 00:00:00-04:00  182.66  184.22  179.23  181.83  16181942  
2020-11-02 00:00:00-05:00  184.51  186.92  183.89  184.68  13860489  
2020-11-03 00:00:00-05:00  186.83  189.53  185.16  188.14  11004582  

[5 rows x 50 columns]

In [12]:
# Create and empty DataFrame for closing prices for "SQ","ADYEY","MELI","GDOT","PYPL","FISV","PFE","GS","V","AXP","SPY"
df_closing_prices = pd.DataFrame()

# Fetch the closing prices of "SQ","ADYEY","MELI","GDOT","PYPL","FISV","GS","V","AXP","SPY"
df_closing_prices["AAPL"] = df_portfolio_year["AAPL"]["close"]
df_closing_prices["MSFT"] = df_portfolio_year["MSFT"]["close"]
df_closing_prices["AMZN"] = df_portfolio_year["AMZN"]["close"]
df_closing_prices["GOOG"] = df_portfolio_year["GOOG"]["close"]
df_closing_prices["FB"] = df_portfolio_year["FB"]["close"]
df_closing_prices["TSLA"] = df_portfolio_year["TSLA"]["close"]
df_closing_prices["BRK.A"] = df_portfolio_year["BRK.A"]["close"]
df_closing_prices["V"] = df_portfolio_year["V"]["close"]
df_closing_prices["JPM"] = df_portfolio_year["JPM"]["close"]
df_closing_prices["JNJ"] = df_portfolio_year["JNJ"]["close"]

# Drop the time component of the date
df_closing_prices.index = df_closing_prices.index.date

In [13]:
# Compute daily returns after cleaning the null values if any
df_daily_returns = df_closing_prices.pct_change().dropna()